$\large\textbf{Exercise 2.}$



In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer as timer
np.random.seed(10)

In [2]:
def evalf(x, n, lamda):
  return 0.5 * (np.linalg.norm(np.subtract(np.matmul(A,x) , y)))**2 + lamda * (np.linalg.norm(x)**2) * 0.5

In [3]:
def evalg(x, n, lamda):
  return np.add(np.matmul(A.T, np.subtract(np.matmul(A, x), y)), lamda * x)

In [4]:
def evalh(x, n, lamda):
  return np.add(np.matmul(A.T,A), lamda * np.identity(n))

In [5]:
def compute_B_k(s, y, B_k, n, lamda):
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  I = np.identity(n)
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [6]:
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [7]:
def compute_D_k(x,n, lamda):
  mat = evalh(x, n, lamda)
  return np.linalg.inv(mat)

In [8]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, B_k, lamda, n): #add appropriate arguments to the function
  alpha = alpha_start
  p =  -gradf
  D_k = B_k
  while evalf(x + alpha*np.matmul(D_k,p), n, lamda) > evalf(x, n, lamda) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p))):
    alpha = alpha * rho
  return alpha

In [9]:
def find_minimizer_Newtonmethod(start_x, n, tol, line_search_type , lamda, *args):
  x = start_x
  g_x = evalg(x,n, lamda)
  h_x = evalh(x,n, lamda)
  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  step_length = 1.
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    D_k = np.linalg.inv(evalh(x, n, lamda))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma, D_k, lamda, n)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x, n, lamda) #compute gradient at new point
  return x

In [10]:
def find_minimizer_BFGS_scaling(start_x, n, tol, line_search_type, lamda, *args):
  x = start_x
  g_x = evalg(x,n, lamda)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  B_k = np.identity(n)
  x_old = x
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old, g_x, alpha_start, rho, gamma, B_k, lamda, n)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new,n, lamda)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,n, lamda), evalg(x_old,n, lamda))
    B_k = compute_B_k(s, y, B_k, n, lamda)
    x_old = x_new
    k += 1
  return x_new

In [ ]:
#Code for Newton method
my_tol = my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(10)
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1)
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  x_opt = find_minimizer_Newtonmethod(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)
  newtontime = timeit.default_timer() - start
  diff = np.subtract(x_opt, xorig)
  print("\n\nd:", d, "\nTotal time:", newtontime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)



d: 1000 
Total time: 38.187301320023835 
L2 norm difference ||Ax* - y||_2^2:  2.8821151191112977e-07 
L2 norm difference || x_opt - xorig||:  781.2270054785175


d: 5000 
Total time: 122.6255974038504 
L2 norm difference ||Ax* - y||_2^2:  4.254294001142676e-08 
L2 norm difference || x_opt - xorig||:  4798.386398060507


d: 10000 
Total time: 235.82637259969488 
L2 norm difference ||Ax* - y||_2^2:  2.3225014298127347e-08 
L2 norm difference || x_opt - xorig||:  9774.842929392476


$\large\textbf{Question 1.}$

$\large\textbf{Answer 1.}$

In the previous exercise, the Hessian matrix was represented as $A^TA$. However, by introducing a regularization term, it transforms into $A^TA + \lambda I$. The invertibility of the matrix $A^TA$ cannot be guaranteed, potentially resulting in the non-existence of the inverse Hessian matrix in the previous exercise.

Conversely, the matrix $A^TA + \lambda I$ is always invertible due to the addition of the regularization term. This is because $A^TA$ is symmetric, and by appending positive diagonal elements to a symmetric matrix, we ensure the positivity of its eigenvalues. Consequently, for $\lambda > 0$, the inverse of the Hessian matrix invariably exists for the regularized function in this exercise.



In [ ]:
#Code for BFGS method
print("Using BFGS Method")
my_tol = my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(1000)
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1)
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  my_start_x = np.zeros((d,1))
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  x_star_bfgs = find_minimizer_BFGS_scaling(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)
  bfgstime = timeit.default_timer() - start
  diff = np.subtract(x_star_bfgs, xorig)
  print("\n\nd:", d, "\nTotal time:", bfgstime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_star_bfgs), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)

Using BFGS Method


d: 1000 
Total time: 5.62871303409338 
L2 norm difference ||Ax* - y||_2^2:  2.1228642386241914e-07 
L2 norm difference || x_opt - xorig||:  817.720298645376


d: 5000 
Total time: 163.23696874128655 
L2 norm difference ||Ax* - y||_2^2:  4.740359614931582e-08 
L2 norm difference || x_opt - xorig||:  4774.91335990538


d: 10000 
Total time: 954.5860014227219 
L2 norm difference ||Ax* - y||_2^2:  1.7521053952366658e-08 
L2 norm difference || x_opt - xorig||:  9825.861490427405


**Question 1**


Newton's Method

Dimention of A | Total Time |  $||Ax^* - y||_2^2$   |  $||x^* - x_{orig}||_2^2$
:----:|:---:|:---:|:---:
(200, 1000) | 38.187301320023835 | 2.8821151191112977e-07 | 781.2270054785175
(200, 5000) | 122.6255974038504 | 4.254294001142676e-08   | 4798.386398060507
(200, 10000) | 235.82637259969488  | 2.3225014298127347e-08 | 9774.842929392476


BFGS Method

Dimention of A | Total Time |  $||Ax^* - y||_2^2$   |  $||x^* - x_{orig}||_2^2$
:----:|:---:|:---:|:---:
(200, 1000) | 5.62871303409338  | 2.1228642386241914e-07   | 817.720298645376
(200, 5000) | 163.23696874128655  | 4.740359614931582e-08  | 4774.91335990538
(200, 10000) | 954.5860014227219  | 1.7521053952366658e-08 | 9825.861490427405

**Question 2**

The failure occures for newton's method after size of A = (200, 10000)
The failure occures for BFGS method after size of A = (200, 10000)